Yelp es un sitio donde se pueden dejar reseñas y tips sobre negocios de todo tipo, muy utilizado en Estados Unidos. Los csvs son:

* businesses.csv
  * business_id
  * name
  * address
  * city
  * state
  * postal_code
  * latitude
  * longitude
  * stars: float, star rating, rounded to half-stars
  * review_count
  * is_open
  * attributes: un json con atributos variados
  * categories: lista de categorías en json a las que pertenece el negocio
  * hours: diccionario json con horarios día por día en los que abre
* review.csv
  * review_id
  * user_id
  * business_id
  * stars: entero entre 1 y 5
  * date
  * text: texto de la review
  * useful: cantidad de gente que la voto útil
  * funny: cantidad de gente que la voto divertida
  * cool: cantidad de gente que la votó como cool
* user.csv
  * user_id
  * name
  * review_count
  * yelping_since: fecha de registro
  * friends: lista en json de user ids que son amigos
  * useful: cantidad de votos “useful” que recibió el usuario
  * funny: cantidad de votos “funny” que recibió el usuario
  * cool: cantidad de votos “cool” que recibió el usuario
  * fans: cantidad de fans que tiene el usuario
  * elite: lista json con los años que el usuario fue élite
  * average_stars: promedio de rating de todas sus reviews
  * compliment_*: la cantidad de cumplidos de tipo * que recibió el usuario
* checkins.csv: Las visitas que los usuarios registran a los lugares
  * business_id
  * date: lista de fechas separadas por coma de los chekins que recibió el negocio
* tip.csv: tips que los usuarios dan sobre lugares
  * text
  * date
  * compliment_count: cumplidos que recibió el tip
  * business_id
  * user_id

### C3. Utilizando la declaración universal de los derechos humanos en [español](https://www.ohchr.org/sites/default/files/UDHR/Documents/UDHR_Translations/spn.pdf) y [portugues](https://www.oas.org/dil/port/1948%20Declara%C3%A7%C3%A3o%20Universal%20dos%20Direitos%20Humanos.pdf) y utilizando la distancia normalizada de compresión, encuentre una review en español y otra en portugues (puede tomar un sample siempre y cuando sea capaz de encontrar eso en el sample).

In [3]:
import gzip
import pandas as pd
from tqdm import tqdm
import numpy as np

In [4]:
reviews = pd.read_csv('/content/drive/MyDrive/Datos/TP1/Parte 1/review.csv', usecols=['text']).text.to_list()

archivo_esp = open('/content/drive/MyDrive/Datos/TP1/Parte 1/declaracion_esp.txt', 'r')
declaracion_esp = archivo_esp.read()
archivo_esp.close()

archivo_por = open('/content/drive/MyDrive/Datos/TP1/Parte 1/declaracion_por.txt', 'r')
declaracion_por = archivo_por.read()
archivo_por.close()

In [5]:
len(reviews)

3838105

Cálculo de la Normalized Compression Distance (NCD):

In [6]:
def compress_size(bytefile):
    return len(gzip.compress(bytefile))

def ncd(obj1, obj2):
    len1 = compress_size(obj1)
    len2 = compress_size(obj2)
    concat_len = len(gzip.compress(obj1+obj2))
    return (concat_len-min(len1, len2)) / max(len1, len2)

Busco una review en español en un sample de 500.000 reviews:

In [7]:
(review_esp, distancia_min) = ('', 100.0)

for review in tqdm(reviews[:500000]):
  distancia = ncd(review.encode(), declaracion_esp.encode())
  if distancia < distancia_min:
    (review_esp, distancia_min) = (review, distancia)

review_esp

100%|██████████| 500000/500000 [10:04<00:00, 827.71it/s]


'He cenado en este restaurante varias veces, y cada vez he salido con un buen sabor en la boca, literalmente y figurativamente. Desde el momento en que entras, es como si fueras transportado a un restaurante auténtico en México. Besito juega música latina que no se distrae de su conversación, pero en cambio la música agrega al ambiente. Hay varias zonas para sentarse, además de un bar. \n ¡El servicio es excepcional! Los camareros vienen a su mesa a menudo para asegurarse de que su experiencia está por encima del promedio. Los meseros son de buen humor y pueden responder a cualquier pregunta que tengas sobre el menú. Hay dos platos deliciosos que yo recomiendo. El primer plato es los tacos de carne asada que están llenos de queso Chihuahua derretido, pico de gallo, salsa de aguacate verde, encima de una falda de ternera. A mí me gusta este plato principal puesto que viene con frijoles negros y arroz. El segundo plato que recomiendo es las enchiladas suizas. Estas enchiladas están llena

Busco una review en portugués en un sample de 500.000 reviews:

In [8]:
(review_por, distancia_min) = ('', 100.0)

for review in tqdm(reviews[:500000]):
  distancia = ncd(review.encode(), declaracion_por.encode())
  if distancia < distancia_min:
    (review_por, distancia_min) = (review, distancia)

review_por

100%|██████████| 500000/500000 [09:29<00:00, 877.59it/s]


'Não compro no mercado brasileiro com frequência, porém cada vez vejo menos variedade e menos empenho por parte dos funcionários quanto ao atendimento. Alguns funcionários não são receptivos, não se colocam à disposição, e não falam Inglês ou Português. Não sei quais são as condições de trabalho, mas como consumidora tenho menos interesse.'

### C8. Yelp tiene la teoría de que mientras más variadas y “ricas” son las reviews de un usuario más popular es. Mientras más variadas son las reviews de un usuario, más difíciles son de comprimir. Por otro lado, mientras más textos se comprimen más fácil es lograr un buen ratio de compresión por lo que vamos a quedarnos con los usuarios que hicieran más de 200 reviews. ¿Cómo se correlaciona el ratio de compresión de todas las reviews de cada uno de esos usuarios con su cantidad de fans?

In [9]:
import gzip
import pandas as pd
from tqdm import tqdm
import numpy as np

In [10]:
reviews = pd.read_csv('/content/drive/MyDrive/Datos/TP1/Parte 1/review.csv', usecols=['text', 'user_id'])
users = pd.read_csv('/content/drive/MyDrive/Datos/TP1/Parte 1/user.csv', usecols=['user_id', 'review_count', 'fans'])
users

,user_id,review_count,fans
0,qVc8ODYU5SZjKXVBgXdI7w,5.0,267
1,j14WgRoU_-2ZE1aw1dXrJg,64.0,3138
2,2WnXYQFK0hXEoTxPtV2zvg,0.0,52
3,SZDeASXq7o05mMNLshsdIA,0.0,28
4,hA5lMy-EnncsH4JoR-hFGQ,7.0,1
...,...,...,...
1987892,fB3jbHi3m0L2KgGOxBv6uw,1.0,0
1987893,68czcr4BxJyMQ9cJBm6C7Q,1.0,0
1987894,1x3KMskYxOuJCjRz70xOqQ,1.0,0
1987895,ulfGl4tdbrH05xKzh5lnog,1.0,0


Me quedo con los usuarios con más de 200 reviews:

In [11]:
users = users[users['review_count'] > 200].drop(columns=['review_count'])
users

,user_id,fans
630,FlXBpK_YZxLo27jcMdII1w,102
927,_BcWyKQL16ndpBdggh2kNA,558
1206,Xw7ZjaGfr0WNVt6s_5KZfA,379
1242,pou3BbKsIozfH50rxmnMew,914
1248,Er_1k4cl0VwKz9tfIn-FBw,100
...,...,...
298393,_8_xgsROsH9h53_YErsZ3g,5
303165,6EHUYIyEb-biAOAGRBqEpg,8
304643,EnFRwa9xPq9QaE4jBjFhlw,10
305087,qzquNJ5YJiidnry19rdN0w,5


In [12]:
reviews

,user_id,text
0,mh_-eMZ6K5RLWhZyISBhwA,"If you decide to eat here, just be aware it is..."
1,bcjbaE6dDog4jkNY91ncLQ,Cute interior and owner (?) gave us tour of up...
2,RreNy--tOmXMl1en0wiBOg,I was really between 3 and 4 stars for this on...
3,Jha0USGDMefGFRLik_xFQg,First time there and it was excellent!!! It fe...
4,4hBhtCSgoxkrFgHa4YAD-w,"Great burgers,fries and salad! Burgers have a..."
...,...,...
3838100,_XVdmFWSgTN6YlojUxixTA,We redesigned my moms dress and mad it complet...
3838101,Zo0th2m8Ez4gLSbHftiQvg,"This spot offers a great, affordable east week..."
3838102,mm6E4FbCMwJmb7kPDZ5v2Q,This Home Depot won me over when I needed to g...
3838103,YwAMC-jvZ1fvEUum6QkEkw,For when I'm feeling like ignoring my calorie-...


Mergeo los datasets `users` y `reviews`. Como ambos contienen una columna de nombre user_id, la operación se realizará con esa columna. 

In [13]:
reviews = pd.merge(users, reviews)
reviews

,user_id,fans,text
0,FlXBpK_YZxLo27jcMdII1w,102,I don't think I've ever had such bland Asian f...
1,FlXBpK_YZxLo27jcMdII1w,102,The bagels here are good. They don't blow me a...
2,FlXBpK_YZxLo27jcMdII1w,102,Boyfriend and I had a lovely date night at Par...
3,FlXBpK_YZxLo27jcMdII1w,102,Naf Naf! Fun to say and fun to eat! I love ho...
4,FlXBpK_YZxLo27jcMdII1w,102,Boyfriend and I had a great meal here yesterda...
...,...,...,...
87919,leMOR7VSm5z-0r60iJ90EA,3,They keep records of the treatments for the ho...
87920,leMOR7VSm5z-0r60iJ90EA,3,"The food is amazing. I liked the fries, nachos..."
87921,leMOR7VSm5z-0r60iJ90EA,3,I love everything this place is about. It alwa...
87922,leMOR7VSm5z-0r60iJ90EA,3,The ladies here have a bad attitude. I'm reall...


Transformo el dataframe para que cada user_id aparezca una sola vez y la columna text contenga todas sus reviews concatenadas. 

In [14]:
reviews = reviews.groupby('user_id').agg({'text' : lambda x: '; '.join(x)}).reset_index()
users = pd.merge(reviews, users)
users

,user_id,text,fans
0,-G7Zkl1wIWBBmD0KRy_sCw,As a facility to hold a large luncheon meeting...,703
1,-dgC1iRp_JqBTmn5d1wRhg,Schlafly is a St. Louis gem with local brewing...,30
2,-kLVfaJytOJY2-QdQoCcNQ,Fantastic Bloody Mary.\n\nI had to update my r...,669
3,-rBMUsCpWE3103q6ombunA,WOW there's a reason this place is so highly r...,13
4,0DB3Irpf_ETVXu_Ou9vPow,Cash only; it's $35 per person 6-item fixed me...,38
...,...,...,...
279,z1-wLzYSLC7F7QnkeAQoOw,If you're reading this review you're most like...,55
280,zShZM_7gg8cQ8RqNteEPgQ,I just placed a large catering order with this...,35
281,zUk_Ww2q1At1QSyRbUjIGQ,Great Groupon deal & high hopes. \n\nMy salad ...,112
282,zYFGMy1_thjMnvQLX6JNBw,After breakfast we were searching for a park o...,247


Calculo el ratio de compresión para cada usuario como `tamaño sin comprimir / tamaño comprimido`:

In [15]:
def compression_ratio(texts):
  return len(texts) / len(gzip.compress(texts.encode()))

users['compression_ratio'] = users.text.map(compression_ratio)
users

,user_id,text,fans,compression_ratio
0,-G7Zkl1wIWBBmD0KRy_sCw,As a facility to hold a large luncheon meeting...,703,2.555117
1,-dgC1iRp_JqBTmn5d1wRhg,Schlafly is a St. Louis gem with local brewing...,30,2.803028
2,-kLVfaJytOJY2-QdQoCcNQ,Fantastic Bloody Mary.\n\nI had to update my r...,669,2.721992
3,-rBMUsCpWE3103q6ombunA,WOW there's a reason this place is so highly r...,13,2.783311
4,0DB3Irpf_ETVXu_Ou9vPow,Cash only; it's $35 per person 6-item fixed me...,38,2.729360
...,...,...,...,...
279,z1-wLzYSLC7F7QnkeAQoOw,If you're reading this review you're most like...,55,2.615687
280,zShZM_7gg8cQ8RqNteEPgQ,I just placed a large catering order with this...,35,2.898891
281,zUk_Ww2q1At1QSyRbUjIGQ,Great Groupon deal & high hopes. \n\nMy salad ...,112,2.416115
282,zYFGMy1_thjMnvQLX6JNBw,After breakfast we were searching for a park o...,247,2.695276


In [16]:
corr = users.compression_ratio.corr(users.fans)
print("La correlación enrte el ratio de compresión de las reviews de los usuarios con su cantidad de fans es: " + str(corr))

La correlación enrte el ratio de compresión de las reviews de los usuarios con su cantidad de fans es: -0.19343151111932297


A menor ratio de compresión, más difícil es comprimir las reviews del usuario, lo que nos indica que sus reviews son más variadas. 

Como la correlación entre el ratio de compresión de las reviews de un usuario y su cantidad de fans es negativa, confirma que a menor ratio de compresión (reviews más variadas), mayor es la cantidad de fans del usuario (en un 19,34%).